In [15]:
!pip -q install ultralytics

from ultralytics import YOLO

import numpy as np
from numpy import asarray
from PIL import Image

import cv2

In [16]:
# Methods order_points and four_point_transform by Shai Nisan

def order_points(pts):

  # order a list of 4 coordinates:
  # 0: top-left,
  # 1: top-right
  # 2: bottom-right,
  # 3: bottom-left

  rect = np.zeros((4, 2), dtype = "float32")
  s = pts.sum(axis = 1)
  rect[0] = pts[np.argmin(s)]
  rect[2] = pts[np.argmax(s)]

  diff = np.diff(pts, axis = 1)
  rect[1] = pts[np.argmin(diff)]
  rect[3] = pts[np.argmax(diff)]

  return rect

def four_point_transform(image, pts):

  img = Image.open(image)

  # account for images of non-RGB modes (e.g. grayscale, palette)
  img = img.convert("RGB")

  image = np.asarray(img)
  (tl, tr, br, bl) = pts

  # compute the width of the new image
  widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
  widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
  maxWidth = max(int(widthA), int(widthB))


  # compute the height of the new image
  heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
  heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
  maxHeight = max(int(heightA), int(heightB))

  # construct set of destination points to obtain a "birds eye view"
  dst = np.array([
    [0, 0],
    [maxWidth - 1, 0],
    [maxWidth - 1, maxHeight - 1],
    [0, maxHeight - 1]], dtype = "float32")

  # compute the perspective transform matrix and then apply it
  M = cv2.getPerspectiveTransform(pts, dst)
  warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))

  img = Image.fromarray(warped, "RGB")

  return img

def detect_corners(image):

  # Use model to detect corners
  model_trained = YOLO("corners.pt")
  results = model_trained.predict(source=image, conf=0.25, verbose=False)

  # Stores corner coordinates in numpy array
  boxes = results[0].boxes
  arr = boxes.xywh.numpy()
  points = arr[:,0:2]

  # Selects best four corners
  corners = order_points(points)

  return corners

def detect_pieces(image):

  # Use model to detect pieces
  model_trained = YOLO("pieces.pt")
  results = model_trained.predict(source=image, conf=0.25, augment=False, verbose=False)
  w, h = image.size

  # Stores piece coordinates and piece type in separate numpy arrays
  boxes = results[0].boxes
  detections = boxes.xyxy.cpu().numpy()
  classes = boxes.cls.cpu().numpy()

  # Takes the center of each bounding box and divides the coordinates
  # by the width and height of the image to determine which square it is in
  coords = []
  for box in detections:
    arr = [int((box[0]+box[2])*4/w), int((box[1]+box[3])*4/h)]
    coords.append(arr)

  # Converts class into FEN letter
  identifiers = []
  ref = ["B", "K", "N", "P", "Q", "R", "b", "k", "n", "p", "q", "r"]
  for i in classes:
    identifiers.append(ref[int(i)])

  return coords, identifiers

def to_FEN_array(coordinates, identifier):

  # Reverse chess piece array to sort in increasing confidence
  coordinates = coordinates[::-1]
  identifier = identifier[::-1]

  # Initialize empty 8 by 8 array
  final_FEN_array = [[None for x in range(8)] for y in range(8)]

  # Assigns FEN letters to their coordinates on the grid. Lowest confidence first so that
  # overlapping false positives are replaced by the correct piece
  for i in range(len(coordinates)):
    final_FEN_array[coordinates[i][1]][coordinates[i][0]] = identifier[i]

  return final_FEN_array

def to_FEN(final_FEN_array):

  final_FEN = ""

  # Read FEN array row by row to construct FEN string
  for line in final_FEN_array:
    temp_num = 0
    for FEN_identifier in line:
      if isinstance(FEN_identifier,str):
        if temp_num == 0:
          final_FEN += FEN_identifier
        else:
          final_FEN += str(temp_num) + FEN_identifier
          temp_num = 0
      else:
        temp_num += 1
    if temp_num == 0:
      final_FEN += "/"
    else:
      final_FEN += str(temp_num) + "/"

  final_FEN = final_FEN[:-1]

  return final_FEN

In [17]:
# In this repository is an example image of the chessboard we trained with.
image = "placeholder.png"

corners = detect_corners(image)
transformed_image = four_point_transform(image, corners)
coords, identifiers = detect_pieces(transformed_image)
final_FEN_array = to_FEN_array(coords,identifiers)
final_FEN = to_FEN(final_FEN_array)
print("https://lichess.org/analysis/" + final_FEN)



https://lichess.org/analysis/4q3/8/5r2/3r2k1/2R1K3/5P1R/8/8
